In [44]:
from langgraph.graph import StateGraph,END,START
from typing import TypedDict,Dict
from langgraph.checkpoint.memory import InMemorySaver

In [45]:
class GraphState(TypedDict,total=False):
    question:str
    answer:str

In [46]:
memory=InMemorySaver()
config={"configurable":{"thread_id":"1"}}
Graph=StateGraph(GraphState)

In [47]:
def router(state:GraphState)->str:
    return state["answer"]

In [48]:

def validate(state:GraphState)->Dict:
    question=state.get("question","").strip()
    answer="1"
    if not question:
        answer="0"
    return {"question":question,"answer":answer}
def greet(state:GraphState)->Dict:
    return {"question":state.get("question"),"answer":"hello you are in valid state"}

In [40]:
Graph.add_node("validate",validate)
Graph.add_node("greet",greet)
Graph.add_conditional_edges("validate",router,{"0":END,"1":"greet"})
Graph.add_edge(START,"validate")
Graph.add_edge("greet",END)

In [41]:
app=Graph.compile(checkpointer=memory)

In [42]:
inputs = {"question": "Hello, how are you?"}
result = app.invoke(inputs,config)
print(result)

{'question': 'Hello, how are you?', 'answer': 'hello you are in valid state'}


In [43]:
graph = app.get_graph()
mermaid_code = graph.draw_mermaid()

print(mermaid_code)


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	validate(validate)
	greet(greet)
	__end__([<p>__end__</p>]):::last
	__start__ --> validate;
	validate -. &nbsp;0&nbsp; .-> __end__;
	validate -. &nbsp;1&nbsp; .-> greet;
	greet --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

